In [1]:
import time, warnings
import pandas as pd
import numpy as np
import rasterio
import os
from joblib import parallel_backend

from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/Users/jonny.sanchez/Documents/tesis/estaciones_final_maestria.csv',sep=";")
df.head()

,fuente,ciudad,codigo_estacion,fecha_toma,anio,mes,dia,medicion,sea_water,fresh_water,...,clouds,bare_ground,veg,lst,ndbi,ndvi,ndwi,st_emissivity,norte,este
0,dimar,barranquilla,0000000004,2020-03-29 07:00:00.000,2020,3,29,26.40,0.04,0.88,...,0.0,0.00,0.0,29.455811,-0.011720,-0.016581,0.002060,0.9880,2.785929e+06,4.798071e+06
1,dimar,barranquilla,0000000004,2021-02-12 13:00:00.000,2021,2,12,28.12,0.08,0.72,...,0.0,0.00,0.0,28.509033,-0.013787,0.060807,-0.105339,0.9880,2.785929e+06,4.798071e+06
2,dimar,barranquilla,0000000004,2022-02-23 13:00:00.000,2022,2,23,27.81,0.25,0.70,...,0.0,0.00,0.0,29.517365,-0.096698,0.052519,-0.084309,0.9880,2.785929e+06,4.798071e+06
3,dimar,barranquilla,0CP03FL033,2020-03-29 13:00:00.000,2020,3,29,28.50,0.00,0.28,...,0.0,0.04,0.0,36.818237,0.073587,0.277937,-0.362358,0.9669,2.778642e+06,4.801240e+06
4,dimar,barranquilla,0CP03FL033,2021-02-12 13:00:00.000,2021,2,12,28.60,0.00,0.32,...,0.0,0.20,0.0,34.975922,0.049616,0.316371,-0.368710,0.9669,2.778642e+06,4.801240e+06


In [3]:
y = df['medicion'].values

bands = ['sea_water','fresh_water','builds','clouds','bare_ground','veg','lst','ndvi','ndbi','norte','este']
available_cols = [c for c in bands if c in df.columns]

X = df[available_cols]

In [4]:
X.corr()

,sea_water,fresh_water,builds,clouds,bare_ground,veg,lst,ndvi,ndbi,norte,este
sea_water,1.000000,-0.121512,-0.526786,NaN,-0.356849,-0.503944,-0.743150,-0.580274,0.090142,0.085052,0.016479
fresh_water,-0.121512,1.000000,-0.057815,NaN,-0.066334,-0.070632,-0.096071,-0.083924,-0.016298,0.021524,-0.031713
builds,-0.526786,-0.057815,1.000000,NaN,-0.042686,-0.277986,0.756813,0.090054,-0.043944,-0.128136,-0.198516
clouds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bare_ground,-0.356849,-0.066334,-0.042686,NaN,1.000000,-0.082940,0.419740,0.195668,-0.029842,0.001348,-0.004891
veg,-0.503944,-0.070632,-0.277986,NaN,-0.082940,1.000000,-0.024201,0.565311,-0.048643,0.010942,0.190711
lst,-0.743150,-0.096071,0.756813,NaN,0.419740,-0.024201,1.000000,0.286439,-0.054163,-0.286621,-0.251972
ndvi,-0.580274,-0.083924,0.090054,NaN,0.195668,0.565311,0.286439,1.000000,-0.208937,0.032984,0.111235
ndbi,0.090142,-0.016298,-0.043944,NaN,-0.029842,-0.048643,-0.054163,-0.208937,1.000000,-0.103873,-0.086823
norte,0.085052,0.021524,-0.128136,NaN,0.001348,0.010942,-0.286621,0.032984,-0.103873,1.000000,0.916934


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    'LinearRegression' : (
        Pipeline([("scaler", StandardScaler()),("model",LinearRegression())]), 
        {}
    ),
    'RandomForestRegressor': (
        Pipeline([("model", RandomForestRegressor(random_state=42,n_jobs=-1))]),
        {"model__n_estimators":[200], 
         "model__max_depth":[None,8],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]}
    ),
    'ExtraTrees':(
        Pipeline([("model",ExtraTreesRegressor(random_state=42,n_jobs=-1))]),
        {
        "model__n_estimators": [300, 500],
        "model__max_depth": [None, 15],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]}
    ),
    'GradientBoostingRegressor': (
        Pipeline([("model", GradientBoostingRegressor(random_state=42))]),
        {"model__n_estimators":[200,400], 
         "model__learning_rate":[0.05,0.1], 
         "model__max_depth":[2,3]}
    ),
    'SVR': (
        Pipeline([("scaler", StandardScaler()), ("model", SVR())]),
        {"model__C": [1.0, 10.0], 
         "model__epsilon": [0.1, 0.3], 
         "model__kernel": ["rbf"]}
    ),
    'MLPRegressor': (
        Pipeline([("scaler", StandardScaler()), ("model", MLPRegressor(random_state=42, max_iter=3000))]),
        {"model__hidden_layer_sizes": [(64, 32), (128, 64, 32)], 
         "model__alpha": [1e-5, 1e-4, 1e-3, 1e-2], 
         "model__activation": ["relu","tanh"],
         "model__solver": ["adam", "lbfgs"],}
    ),
}

In [6]:
k          = 10
cv         = KFold(n_splits=k, shuffle=True, random_state=42)

results = []
best_models = {}

for name, (pipe, grid) in models.items():
    gs = GridSearchCV(pipe, grid, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=1, refit=True)
    tic = time.perf_counter()
    gs.fit(X_train, y_train)
    fit_time = round(time.perf_counter() - tic, 3)

    y_pred = gs.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = float(np.sqrt(mse))
    mae = float(mean_absolute_error(y_test, y_pred))
    r2 = float(r2_score(y_test, y_pred))

    with parallel_backend("threading"):
        tic = time.perf_counter()
        y_pred_oof = cross_val_predict(gs.best_estimator_, X, y, cv=cv, n_jobs=-1)
        cv_pred_time = round(time.perf_counter() - tic, 3)

    rmse_oof = float(np.sqrt(mean_squared_error(y, y_pred_oof)))
    mae_oof  = float(mean_absolute_error(y, y_pred_oof))
    r2_oof   = float(r2_score(y, y_pred_oof))

    print(f"\n=== {name} ===")
    print("Mejores params CV: ", gs.best_params_)
    print(f"Tiempo entrenamiento: {fit_time}s | Tiempo OOF-predict: {cv_pred_time}s")
    print(f"[TEST]   RMSE={rmse:.3f}  MAE={mae:.3f}  R2={r2:.3f}")
    print(f"[OOF-CV] RMSE={rmse_oof:.3f}  MAE={mae_oof:.3f}  R2={r2_oof:.3f}")

    results.append({
        "modelo": name,
        "best_params": gs.best_params_,
        "fit_time_s": fit_time,
        "cv_pred_time_s": cv_pred_time,
        "test_RMSE": rmse, "test_MAE": mae, "test_R2": r2,
        "oof_RMSE": rmse_oof, "oof_MAE": mae_oof, "oof_R2": r2_oof,
        "cv_best_RMSE": float(-gs.best_score_),
    })

    best_models[name] = gs.best_estimator_

res_df = pd.DataFrame(results).sort_values("test_RMSE", ascending=False)
display(res_df)


=== LinearRegression ===
Mejores params CV:  {}
Tiempo entrenamiento: 0.024s | Tiempo OOF-predict: 0.035s
[TEST]   RMSE=14.955  MAE=3.169  R2=-30.948
[OOF-CV] RMSE=6.929  MAE=1.451  R2=-6.775

=== RandomForestRegressor ===
Mejores params CV:  {'model__max_depth': 8, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 200}
Tiempo entrenamiento: 6.657s | Tiempo OOF-predict: 0.766s
[TEST]   RMSE=1.058  MAE=0.806  R2=0.840
[OOF-CV] RMSE=1.007  MAE=0.790  R2=0.836

=== ExtraTrees ===
Mejores params CV:  {'model__max_depth': None, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 500}
Tiempo entrenamiento: 16.751s | Tiempo OOF-predict: 1.208s
[TEST]   RMSE=1.030  MAE=0.769  R2=0.848
[OOF-CV] RMSE=0.972  MAE=0.775  R2=0.847

=== GradientBoostingRegressor ===
Mejores params CV:  {'model__learning_rate': 0.05, 'model__max_depth': 3, 'model__n_estimators': 200}
Tiempo entrenamiento: 4.264s | Tiempo OOF-predict: 0.268s
[TEST]   RM

,modelo,best_params,fit_time_s,cv_pred_time_s,test_RMSE,test_MAE,test_R2,oof_RMSE,oof_MAE,oof_R2,cv_best_RMSE
0,LinearRegression,{},0.024,0.035,14.954765,3.168832,-30.948436,6.928910,1.450639,-6.775246,1.269111
5,MLPRegressor,"{'model__activation': 'tanh', 'model__alpha': ...",269.790,5.327,8.491305,2.216038,-9.300050,3.356308,1.345439,-0.824349,1.255211
4,SVR,"{'model__C': 10.0, 'model__epsilon': 0.1, 'mod...",0.071,0.014,1.342263,0.979232,0.742625,1.170783,0.903665,0.778008,1.099880
3,GradientBoostingRegressor,"{'model__learning_rate': 0.05, 'model__max_dep...",4.264,0.268,1.064832,0.776133,0.838023,1.021615,0.789000,0.830972,1.028142
1,RandomForestRegressor,"{'model__max_depth': 8, 'model__min_samples_le...",6.657,0.766,1.057883,0.805991,0.840131,1.006881,0.789659,0.835812,1.046271
2,ExtraTrees,"{'model__max_depth': None, 'model__min_samples...",16.751,1.208,1.030078,0.768561,0.848424,0.972039,0.775194,0.846979,1.004635


In [7]:
feature_order = list(X.columns)
print("feature_order:", feature_order)

feature_order: ['sea_water', 'fresh_water', 'builds', 'clouds', 'bare_ground', 'veg', 'lst', 'ndvi', 'ndbi', 'norte', 'este']


In [8]:
def cargar_banda(path):
    with rasterio.open(path) as src:
        return src.read(), src.profile

def grilla_xy(profile):
    T = profile["transform"]
    W = profile["width"]
    H = profile["height"]
    xs = T.c + np.arange(W) * T.a
    ys = T.f + np.arange(H) * T.e
    X, Y = np.meshgrid(xs, ys)
    return X.astype("float32"), Y.astype("float32")

def read_resample(path, ref_profile):
    with rasterio.open(path) as src:
        if (src.width, src.height) != (ref_profile["width"], ref_profile["height"]) or src.transform != ref_profile["transform"]:
            data = src.read(
                out_shape=(1, ref_profile["height"], ref_profile["width"]),
                resampling=rasterio.enums.Resampling.nearest
            )
            return data.astype("float32")
        else:
            return src.read().astype("float32")

def clasificar_rasters(path_raiz, best_models):
    for folder_name in os.listdir(path_raiz):
        folder_path = os.path.join(path_raiz, folder_name)
        if os.path.isdir(folder_path):
            date = anio = mes = dia = sea_water = fresh_water = builds = clouds = bare_ground = veg = lst = ndbi = ndvi = ndwi = st_emissivity = None
            date = folder_name.split(sep='_')[3]
            anio_file = int(date[0:4])
            mes_file = int(date[4:6])
            dia_file = int(date[6:8])
            for file_name in os.listdir(folder_path):
                if file_name.endswith("c0_sea_water.TIF"):
                    sea_water = os.path.join(folder_path,file_name)
                elif file_name.endswith("c1_fresh_water.TIF"):
                    fresh_water = os.path.join(folder_path,file_name)
                elif file_name.endswith("c2_builds.TIF"):
                    builds = os.path.join(folder_path,file_name)
                elif file_name.endswith("c3_clouds.TIF"):
                    clouds = os.path.join(folder_path,file_name)
                elif file_name.endswith("c4_bare_ground.TIF"):
                    bare_ground = os.path.join(folder_path,file_name)
                elif file_name.endswith("c5_vegetation.TIF"):
                    veg = os.path.join(folder_path,file_name)
                elif file_name.endswith("LST.TIF"):
                    lst = os.path.join(folder_path,file_name)
                elif file_name.endswith("NDBI.TIF"):
                    ndbi = os.path.join(folder_path,file_name)
                elif file_name.endswith("NDVI.TIF"):
                    ndvi = os.path.join(folder_path,file_name)
                elif file_name.endswith("NDWI.TIF"):
                    ndwi = os.path.join(folder_path,file_name)
                elif file_name.endswith("ST_EMIS.TIF"):
                    st_emissivity = os.path.join(folder_path,file_name)

        # cargar y apilar
            sea_water, _ = cargar_banda(sea_water)
            fresh_water, _ = cargar_banda(fresh_water)
            builds, _ = cargar_banda(builds) 
            clouds, _ = cargar_banda(clouds) 
            bare_ground, _ = cargar_banda(bare_ground) 
            veg, _ = cargar_banda(veg) 
            lst, profile = cargar_banda(lst) 
            ndbi, _ = cargar_banda(ndbi) 
            ndvi, _ = cargar_banda(ndvi) 
            ndwi, _ = cargar_banda(ndwi) 
            st_emissivity, _ = cargar_banda(st_emissivity)

            assert sea_water.shape[1:] == fresh_water.shape[1:] == builds.shape[1:] == clouds.shape[1:] == bare_ground.shape[1:] == veg.shape[1:] == lst.shape[1:] == ndbi.shape[1:] == ndwi.shape[1:] == ndvi.shape[1:] == st_emissivity.shape[1:], "Las dimensiones no coinciden."
            _, H, W = lst.shape
            anio = np.full((1, H, W), anio_file, dtype='float32')
            mes = np.full((1, H, W), mes_file, dtype='float32')
            dia = np.full((1, H, W), dia_file, dtype='float32')
            este, norte = grilla_xy(profile)
            este = este[np.newaxis, :, :]
            norte = norte[np.newaxis, :, :]

            capa = {
                'anio':anio, 'mes':mes, 'dia':dia, 'sea_water':sea_water, 'fresh_water':fresh_water, 'builds':builds, 'clouds':clouds, 'bare_ground':bare_ground, 'veg':veg, 
                'lst':lst, 'ndvi':ndvi, 'ndbi':ndbi, 'ndwi': ndwi, 'st_emissivity':st_emissivity, 'norte':norte, 'este':este,
            }

            arrays = [capa[f] for f in feature_order]
            multiband = np.concatenate((arrays), axis=0)
            num_bands, alto, ancho = multiband.shape

            data = multiband.reshape(num_bands, -1).T
            df_pix = pd.DataFrame(data, columns=feature_order)

            #profile_out = profile.copy()
            #profile_out.update(dtype=rasterio.uint8, count=1, height=int(alto), width=int(ancho))

            for name, model in best_models.items():
                preds = model.predict(df_pix)
                classification = preds.reshape(alto, ancho)

                out_path = os.path.join(folder_path, f"Heat_Islands_P2{name}.TIF")

                profile.update(dtype=rasterio.float32, count=1, height=alto, width=ancho, nodata = np.nan)

                with rasterio.open(out_path, "w", **profile) as dst:
                    dst.write(classification.astype("float32"), 1)
                
                print("Procesado: ", out_path)

In [10]:
path = '/Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta'
clasificar_rasters(path,best_models)

Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/Heat_Islands_P2LinearRegression.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/Heat_Islands_P2RandomForestRegressor.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/Heat_Islands_P2ExtraTrees.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/Heat_Islands_P2GradientBoostingRegressor.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/Heat_Islands_P2SVR.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_009052_20180204_20200902_02_T1/Heat_Islands_P2MLPRegressor.TIF
Procesado:  /Users/jonny.sanchez/Documents/tesis/10-heat_island/santa_marta/LC08_L2SP_00905